In [2]:
# Render our plots inline
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('mode.chained_assignment', None)

In [3]:
log_file = pd.read_csv('log11-18.csv')

In [4]:
log_file[:3]

,Unnamed: 0,id,timestamp,sid,event,act,div_id,course_id,base_course
0,0,1760,2017-08-13 20:23:59,301304554400186198747097533764849583054@208.15...,page,view,index.html,47.0,apcsareview
1,1,1778,2017-08-13 23:14:25,del1,page,view,index.html,29.0,apcsareview
2,2,1779,2017-08-13 23:17:48,del1,page,view,VariableBasics/operators.html,29.0,apcsareview


In [5]:
log_file = log_file.loc[log_file['base_course'].str.startswith('StudentCSP', na=False)]

In [6]:
pretest_logs = log_file.loc[log_file['div_id'].str.startswith('pre_', na=False)] 

In [7]:
pretest_logs['pretest_score'] = np.where(pretest_logs['act'].str.endswith('correct', na=False), 1, 0)

In [8]:
pretest_logs.head()

,Unnamed: 0,id,timestamp,sid,event,act,div_id,course_id,base_course,pretest_score
305,305,10247,2017-08-18 20:42:42,271926861551350033289774502469454480334@146.16...,mChoice,answer:1:correct,pre_1,213.0,StudentCSP,1
306,306,10248,2017-08-18 20:42:42,271926861551350033289774502469454480334@146.16...,mChoice,answer:2:correct,pre_7,213.0,StudentCSP,1
307,307,10249,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:0:no,pre_5,213.0,StudentCSP,0
308,308,10250,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:0:no,pre_4,213.0,StudentCSP,0
309,309,10251,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:2:correct,pre_6,213.0,StudentCSP,1


In [10]:
#Interesting case of repeated logs, look also for 10 who has skipped a whole test and answered another one
# pretest_logs.loc[pretest_logs['sid'] == '116558589813929226898517399943430489038@137.205.1.152']
# if we wanted to have skippers vs not skippers
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.filter.html
# pretest_logs['skipped'] = np.where(pretest_logs['act'].str.endswith('answer:undefined:no', na=False), True, False)
# pretest_logs['not_skipped'] = np.where(pretest_logs['act'].str.endswith('answer:undefined:no', na=False), False, True)
# student_scores_g = pretest_logs.groupby(['sid'])
# skippers = student_scores_g.filter(lambda x: x['skipped'].any())
# non_skippers = student_scores_g.filter(lambda x: x['not_skipped'].all())
# skippers_unique_scores = skippers.groupby(['sid','div_id'])['pretest_score'].min().reset_index()
# non_skippers_unique_scores = non_skippers.groupby(['sid','div_id'])['pretest_score'].min().reset_index()
# non_skipper_score_sum = non_skippers_unique_scores.groupby(['sid'])['pretest_score'].sum().reset_index()
# skipper_score_sum = skippers_unique_scores.groupby(['sid'])['pretest_score'].sum().reset_index()
# [non_skipper_score_sum.size, skipper_score_sum.size]

In [50]:
# Create groups of pretest scores by students
students_unique_scores = pretest_logs.groupby(['sid','div_id'])['pretest_score'].min().reset_index()
s_total_score = students_unique_scores.groupby(['sid'])['pretest_score'].sum().reset_index()
score_groups = {}
group_scores = [6,12]
for score in group_scores:
    score_groups[score] = s_total_score["sid"][(s_total_score["pretest_score"] >= score - 6) & (s_total_score["pretest_score"]  < score)]
not_pre_test = set(log_file["sid"])
for score in group_scores:
     not_pre_test = not_pre_test - set(score_groups[score])
group_scores.append('no_pretest')
score_groups['no_pretest'] = pd.Series(list(not_pre_test))

In [48]:
#examns
exams_logs = log_file.loc[log_file['div_id'].str.startswith('e24', na=False)]

In [85]:
exams_logs['exam_name'] = exams_logs['div_id'].str.split('_').str[1]
exams_logs['exam_score'] = np.where(exams_logs['act'].str.endswith('correct', na=False), 1, 0)
exams_logs_unique_scores = exams_logs.groupby(['sid','div_id','exam_name'])['exam_score'].min().reset_index()
e_total_score = exams_logs_unique_scores.groupby(['exam_name','sid'])['exam_score'].sum().reset_index()
exam_names = ['1','2','3','4']
exam_score_groups = {}
exam_group_scores = [6,12]
for e_name in exam_names:
    for e_score in exam_group_scores:
        exam_score_groups[e_name, e_score] = e_total_score["sid"][(e_total_score["exam_score"] >= e_score - 6) 
                                                         & (e_total_score["exam_score"]  < e_score)
                                                         & e_total_score["exam_name"].str.startswith(e_name, na=False)]
for e_name in exam_names:
    not_exams = set(log_file["sid"])
    for e_score in exam_group_scores:
        not_exams = not_exams - set(exam_score_groups[e_name, e_score])
    exam_score_groups[e_name,'no_exam'] = pd.Series(list(not_exams))
exam_group_scores.append('no_exam')

In [97]:
#parsons
parsons_logs = log_file.loc[log_file['event'].str.endswith('parsons', na=False)
                            # only the div that has a number should be included
                            & log_file['div_id'].str.match('[0-9]*_', na=False)
                           ]
parsons_logs['parsons_score'] = np.where(parsons_logs['act'].str.startswith('correct', na=False), 1, 0)
parsons_logs['parsons_chapter'] = parsons_logs['div_id'].str.split('_').str[0]
parsons_logs_unique_scores = parsons_logs.groupby(['sid','div_id','parsons_chapter'])['parsons_score'].min().reset_index()

In [98]:
p_score_groups = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            p_score_groups[score,e_name, e_score] = parsons_logs_unique_scores[ parsons_logs_unique_scores['sid'].isin(score_groups[score])
                                                              & parsons_logs_unique_scores['sid'].isin(exam_score_groups[e_name, e_score])
                                                              ]
p_score_totals = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            p_score_totals[score,e_name, e_score] =  p_score_groups[score,e_name, e_score].groupby('parsons_chapter')['parsons_score'].agg(['sum','count']).reset_index()

In [99]:
#multiple_choice
mc_logs = log_file.loc[log_file['event'].str.endswith('mChoice', na=False) 
                            & log_file['div_id'].str.match('[0-9]*_', na=False)
                           ]
mc_logs['mc_score'] = np.where(mc_logs['act'].str.endswith('correct', na=False), 1, 0)
mc_logs['mc_chapter'] = mc_logs['div_id'].str.split('_').str[0]
mc_logs_unique_scores = mc_logs.groupby(['sid','div_id','mc_chapter'])['mc_score'].min().reset_index()

In [100]:
m_c_score_groups = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            m_c_score_groups[score,e_name, e_score] = mc_logs_unique_scores[ mc_logs_unique_scores['sid'].isin(score_groups[score])
                                                        & mc_logs_unique_scores['sid'].isin(exam_score_groups[e_name, e_score])
                                                    ]
m_c_score_totals = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            m_c_score_totals[score,e_name, e_score] =  m_c_score_groups[score,e_name, e_score].groupby('mc_chapter')['mc_score'].agg(['sum','count']).reset_index()

In [101]:
#unit test
ut_logs = log_file.loc[log_file['event'].str.endswith('unittest', na=False)]
ut_logs['ut_score'] = np.where(ut_logs['act'].str.endswith('failed:1', na=False), 0, 1)
ut_logs['ut_chapter'] = ut_logs['div_id'].str.split('_').str[1]
ut_logs_unique_scores = ut_logs.groupby(['sid','div_id','ut_chapter'])['ut_score'].min().reset_index()

In [102]:
ut_score_groups = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            ut_score_groups[score,e_name, e_score] = ut_logs_unique_scores[ ut_logs_unique_scores['sid'].isin(score_groups[score])
                                                        & ut_logs_unique_scores['sid'].isin(exam_score_groups[e_name, e_score])
                                                    ]
ut_score_totals = {}
for e_name in exam_names:
    for e_score in exam_group_scores:
        for score in group_scores:
            ut_score_totals[score,e_name, e_score] =  ut_score_groups[score,e_name, e_score].groupby('ut_chapter')['ut_score'].agg(['sum','count']).reset_index()

In [103]:
exam_arrays = {}
for e_name in exam_names:
    group_count = 0
    exam_array = [['combo_id','pretest','exam2','student_count','event','div','percent_correct']]
    for e_score in exam_group_scores:
        for score in group_scores:
            group_count = group_count + 1
            for div in m_c_score_totals[score,e_name, e_score].values:
                exam_array.append([group_count,score,e_score,div[2],'mchoice',div[0], div[1]/div[2] * 100])
            for div in p_score_totals[score,e_name, e_score].values:
                exam_array.append([group_count,score,e_score,div[2],'parsons',div[0], div[1]/div[2] * 100])
            for div in ut_score_totals[score,e_name, e_score].values:
                exam_array.append([group_count,score,e_score,div[2],'unittest',div[0], div[1]/div[2] * 100])
    exam_arrays[e_name] = exam_array

In [104]:
for key in exam_arrays:
    a = np.asarray(exam_arrays[key])
    np.savetxt("exam" + key + ".csv", a, delimiter=",",  fmt='%s')

,div_id,sum,count
0,10_1_2_Rectangle,63,280
1,10_2_2_Triangle,123,235
2,10_3_1_Turtle_Spiro,45,239
3,10_3_2_Turtle_Spiro_Blue_Red,98,194
4,10_4_1_turtle-stamp,13,204
5,10_4_2_turtle-circle,63,188
6,10_5_1_turtle-x,24,188
7,10_5_2_turtle-squares,23,158
8,12_3_2_Price_By_Weight,153,416
9,12_7_1_Even_Odd,172,265


sid                                                      skipped
.mpegasus                                                False       2
100                                                      False       4
100141153404816795579535188407282254798@69.255.125.35    False      10
101166874263566502338341270200684594126@207.238.139.2    False       1
101262661645038672184116268400260629454@168.187.165.77   False       9
101630450808278395732398414816340034510@178.86.241.11    False       8
1025859                                                  False       1
1048                                                     False       9
1090                                                     False       5
1091                                                     False       0
109770308493982268867051443969448952782@204.108.109.107  False      10
109871466372843502234798209890171175886@137.205.1.140    False      10
109913831543735069143626372138844505038@87.38.123.5      False       0
1119        

In [42]:
students_total_score.head(50)

sid
.mpegasus                                                   18.181818
100                                                          0.000000
100141153404816795579535188407282254798@69.255.125.35       90.909091
101166874263566502338341270200684594126@207.238.139.2        9.090909
101262661645038672184116268400260629454@168.187.165.77      81.818182
101630450808278395732398414816340034510@178.86.241.11       72.727273
1025859                                                      9.090909
1048                                                        81.818182
104813990492753513044724130210587558862@168.9.38.4           0.000000
105781639377311585094026025284145464270@168.9.35.17          0.000000
106920695655384831772370930450577966030@121.6.133.134        0.000000
1090                                                        45.454545
1091                                                         0.000000
109770308493982268867051443969448952782@204.108.109.107     90.909091
1098714663728435